In [1]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_datasets
from transformers import *
import pandas as pd
import numpy as np
from tensorflow import keras

from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tqdm.notebook import tqdm
# from wandb.keras import WandbCallback
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
# !wget https://raw.githubusercontent.com/wangz10/contrastive_loss/master/losses.py
import losses
import feed
from feed import *
print('feeding is imported')
import time
import model
from model import *
import sys

from sklearn.preprocessing import LabelEncoder

2.2.0


wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


feeding is imported
2.2.0


In [2]:
train_df = pd.read_csv('files/train.csv')
val_df = pd.read_csv('files/val.csv')
test_df = pd.read_csv('files/test.csv')
pred_df = pd.read_csv('files/pred.csv')
Num_class = len(set(train_df.label.value_counts()))
print(Num_class)

6


In [4]:
BATCH_SIZE = 32
MAX_SEQ_LEN = 128
REPEAT = 2
NUM_TRAIN_ITERATION = REPEAT * int(len(train_df) / BATCH_SIZE)
NUM_TEST_ITERATION = int(len(test_df) / BATCH_SIZE)
print(NUM_TRAIN_ITERATION)
print(NUM_TEST_ITERATION)

15
15


In [5]:
import tensorflow as tf
from transformers import AlbertTokenizer, TFAlbertForSequenceClassification

tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
model = TFAlbertForSequenceClassification.from_pretrained('albert-base-v2', num_labels=Num_class)
# model = TFAlbertModel.from_pretrained('albert-base-v2')

Some weights of the model checkpoint at albert-base-v2 were not used when initializing TFAlbertForSequenceClassification: ['predictions']
- This IS expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFAlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['dropout_3', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
train_dataset = create_dataset('files/train.csv',batch_size=BATCH_SIZE, 
                               tokenizer=tokenizer, max_seq_len=MAX_SEQ_LEN )

val_dataset = create_dataset('files/val.csv',batch_size=BATCH_SIZE, 
                               tokenizer=tokenizer, max_seq_len=MAX_SEQ_LEN )

test_dataset = create_dataset('files/test.csv',batch_size=BATCH_SIZE, 
                               tokenizer=tokenizer, max_seq_len=MAX_SEQ_LEN )

pred_dataset = create_dataset('files/pred.csv',batch_size=BATCH_SIZE, 
                               tokenizer=tokenizer, max_seq_len=MAX_SEQ_LEN ,
                              isinfer=True)

<BatchDataset shapes: (((32, 128), (32, 128), (32, 128)), (32, 1)), types: ((tf.int32, tf.int32, tf.int32), tf.int32)>


In [7]:
# Prepare training: Compile tf.keras model with optimizer, loss and learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])
# print(model.summary())

In [8]:
# Train and evaluate using tf.keras.Model.fit()
history = model.fit(train_dataset, epochs=2, steps_per_epoch=NUM_TRAIN_ITERATION,
                    validation_data=val_dataset, validation_steps=7)

Epoch 1/2
15/15 [==============================] - 169s 11s/step - loss: 1.2358 - accuracy: 0.4771 - val_loss: 1.1420 - val_accuracy: 0.4598
Epoch 2/2
15/15 [==============================] - 164s 11s/step - loss: 1.0380 - accuracy: 0.5417 - val_loss: 1.0737 - val_accuracy: 0.4821


In [9]:
model.evaluate(test_dataset, steps = NUM_TEST_ITERATION )

15/15 [==============================] - 51s 3s/step - loss: 1.0472 - accuracy: 0.5375


[1.047170877456665, 0.5375000238418579]

In [10]:
prediction = model.predict(pred_dataset, steps=NUM_TEST_ITERATION, verbose=1)
print(np.argmax(prediction[0], axis =1))
len(np.argmax(prediction[0], axis =1))

15/15 [==============================] - 50s 3s/step
